In [1]:
# !pip install -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

import math
import librosa
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import os
import SpeechDownloader
import SpeechGenerator
import SpeechModels
import librosa
import wave
from keras.callbacks import TensorBoard

from keras.models import Sequential
from kapre.time_frequency import Melspectrogram, Spectrogram
from kapre.utils import Normalization2D

from keras.models import Model, load_model

from keras.layers import Input, Activation, Concatenate, Permute, Reshape, Flatten, Lambda, Dot, Softmax
from keras.layers import Add, Dropout, BatchNormalization, Conv2D, Reshape, MaxPooling2D, Dense, CuDNNLSTM, Bidirectional
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras import optimizers

from kapre.time_frequency import Melspectrogram, Spectrogram

import keras
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D

from keras.layers import Activation, Add
from keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras.layers import add, Flatten
# from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D
from keras.optimizers import SGD, Adam
import numpy as np

import tensorflow as tf

import IPython.display as ipd

D:\SoftWare\Anaconda3-5.2.0\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def step_decay(epoch):
    initial_lrate = 0.001
    # 每次变为原来的 0.4    
    drop = 0.4
    # 每隔 10 个epoch 下降一次
#     epochs_drop = 10.0
    epochs_drop = 7.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 4e-5):
        lrate = 4e-5
    print('Changing learning rate to {}'.format(lrate))
    return lrate
lrate = LearningRateScheduler(step_decay)

In [3]:
# Download and prepare all data
gscInfo, nCategs = SpeechDownloader.PrepareGoogleSpeechCmd(version=1, task='12cmd')

Google Speech commands dataset version 1 already exists. Skipping download.
Converting test set WAVs to numpy files


0it [00:00, ?it/s]


Converting training set WAVs to numpy files


0it [00:00, ?it/s]


Done preparing Google Speech commands dataset version 1


In [4]:
print(gscInfo.keys())
print(gscInfo['train'].keys())
print(len(gscInfo['train']['files']))
print(nCategs)

dict_keys(['train', 'test', 'val', 'testREAL'])
dict_keys(['files', 'labels'])
51094
12


In [5]:
sr=16000 #we know this one for google audios
iLen = 16000
trainGen = SpeechGenerator.SpeechGen(gscInfo['train']['files'],
                                     gscInfo['train']['labels'],
                                     shuffle=True)

# handle the fact that number of samples in validation may not be multiple of batch_size with shuffle=True
valGen   = SpeechGenerator.SpeechGen(gscInfo['val']['files'],
                                     gscInfo['val']['labels'],
                                     shuffle=False)

#use batch_size = total number of files to read all test files at once
testGen  = SpeechGenerator.SpeechGen(gscInfo['test']['files'],
                                     gscInfo['test']['labels'],
                                     shuffle=False,
                                     batch_size=len(gscInfo['test']['files']))
testRGen = SpeechGenerator.SpeechGen(gscInfo['testREAL']['files'],
                                     gscInfo['testREAL']['labels'],
                                     shuffle=False,
                                     batch_size=len(gscInfo['testREAL']['files']))
valGen.__len__()

106

In [7]:
audios, classes = valGen.__getitem__(0)
classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
# librosa.output.write_wav('file.wav', audios[10], sr, norm=False)
# plt.plot(audios[10])
# ipd.Audio(audios[10], rate=16000)

In [3]:
# melspecModel = Sequential()
# #Keras 序贯模型
# melspecModel.add(Melspectrogram(n_dft=1024, n_hop=128, input_shape=(1, iLen),
#                          padding='same', sr=sr, n_mels=80,
#                          fmin=40.0, fmax=sr/2, power_melgram=1.0,
#                          return_decibel_melgram=True, trainable_fb=False,
#                          trainable_kernel=False,
#                          name='mel_stft') )

# melspecModel.add(Normalization2D(int_axis=0))
# melspecModel.summary()

In [2]:
# melspec = melspecModel.predict(audios.reshape((-1,1,iLen)))
# #输出预测序列
# melspec.shape
# #LL = np.shape(audios.reshape((-1,1,iLen)))
# #LL.shape
# #print(len(audios.reshape((-1,1,iLen))))
# #print(len(melspec)

In [1]:
# plt.figure(figsize=(17,6))
# plt.pcolormesh(melspec[3,:,:,0])

# plt.title('Spectrogram visualization')
# plt.ylabel('Frequency')
# plt.xlabel('Time')

# plt.show()

In [4]:
# plt.hist(melspec.flatten(), bins=100)

# CNN+BiLSTM+Attention

In [12]:
# self-attention LSTM
model = SpeechModels.AttRNNSpeechModel(nCategs, samplingrate=sr, inputLength=iLen)
model.compile(optimizer='adam',
              loss=['sparse_categorical_crossentropy'],
              metrics=['sparse_categorical_accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16000)        0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 16000)     0           input_1[0][0]                    
__________________________________________________________________________________________________
mel_stft (Melspectrogram)       (None, 80, 125, 1)   1091664     reshape_1[0][0]                  
__________________________________________________________________________________________________
normalization2d_2 (Normalizatio (None, 80, 125, 1)   0           mel_stft[0][0]                   
__________________________________________________________________________________________________
permute_1 

In [14]:
#filepath="model_{epoch:02d}.hdf5"
tensorboard = TensorBoard(log_dir='./boards', histogram_freq=0,
                          write_graph=True,  # 是否存储网络结构图
                          write_grads=True, # 是否可视化梯度直方图
                          write_images=True,# 是否可视化参数
                          embeddings_freq=0, 
                          embeddings_layer_names=None, 
                          embeddings_metadata=None)
earlystopper = EarlyStopping(monitor='val_sparse_categorical_accuracy',
                             patience=7, verbose=1)
checkpointer = ModelCheckpoint("model-NewModelCpu_{epoch:02d}.h5",
                               monitor='val_sparse_categorical_accuracy',
                               verbose=1, save_best_only=True)
results = model.fit_generator(trainGen, validation_data = valGen,
                              epochs=40, use_multiprocessing=True,
                              workers=4,
                              callbacks=[earlystopper, checkpointer,
                                         lrate, tensorboard])

Epoch 1/40
Changing learning rate to 0.001


 127/1325 [=>............................] - ETA: 4:22:08 - loss: 3.0218 - sparse_categorical_accuracy: 0.03 - ETA: 2:11:45 - loss: 3.0097 - sparse_categorical_accuracy: 0.08 - ETA: 1:28:16 - loss: 3.0057 - sparse_categorical_accuracy: 0.13 - ETA: 1:06:31 - loss: 2.9819 - sparse_categorical_accuracy: 0.16 - ETA: 53:27 - loss: 2.9726 - sparse_categorical_accuracy: 0.1812 - ETA: 44:44 - loss: 2.9620 - sparse_categorical_accuracy: 0.19 - ETA: 38:31 - loss: 2.9429 - sparse_categorical_accuracy: 0.20 - ETA: 33:51 - loss: 2.9497 - sparse_categorical_accuracy: 0.20 - ETA: 30:13 - loss: 2.9593 - sparse_categorical_accuracy: 0.20 - ETA: 27:18 - loss: 2.9669 - sparse_categorical_accuracy: 0.20 - ETA: 24:56 - loss: 2.9514 - sparse_categorical_accuracy: 0.21 - ETA: 22:57 - loss: 2.9551 - sparse_categorical_accuracy: 0.21 - ETA: 21:16 - loss: 2.9472 - sparse_categorical_accuracy: 0.21 - ETA: 19:50 - loss: 2.9383 - sparse_categorical_accuracy: 0.22 - ETA: 18:35 - loss: 2.9327 - sparse_categorical_ac

 128/1325 [=>............................] - ETA: 2:13:36 - loss: 0.2171 - sparse_categorical_accuracy: 0.93 - ETA: 1:08:03 - loss: 0.2891 - sparse_categorical_accuracy: 0.91 - ETA: 46:08 - loss: 0.3191 - sparse_categorical_accuracy: 0.9115 - ETA: 34:58 - loss: 0.3455 - sparse_categorical_accuracy: 0.89 - ETA: 28:19 - loss: 0.3568 - sparse_categorical_accuracy: 0.89 - ETA: 23:52 - loss: 0.3415 - sparse_categorical_accuracy: 0.89 - ETA: 20:41 - loss: 0.3323 - sparse_categorical_accuracy: 0.89 - ETA: 18:17 - loss: 0.3468 - sparse_categorical_accuracy: 0.89 - ETA: 16:25 - loss: 0.3314 - sparse_categorical_accuracy: 0.90 - ETA: 14:55 - loss: 0.3359 - sparse_categorical_accuracy: 0.90 - ETA: 13:45 - loss: 0.3218 - sparse_categorical_accuracy: 0.90 - ETA: 12:43 - loss: 0.3278 - sparse_categorical_accuracy: 0.90 - ETA: 11:51 - loss: 0.3341 - sparse_categorical_accuracy: 0.90 - ETA: 11:06 - loss: 0.3307 - sparse_categorical_accuracy: 0.90 - ETA: 10:28 - loss: 0.3260 - sparse_categorical_accura

1325/1325 [==============================] - ETA: 1s - loss: 0.2925 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2924 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2924 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2923 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2922 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2921 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2920 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2921 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2920 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2920 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2919 - sparse_categorical_accuracy: 0.909 - ETA: 1s - loss: 0.2919 - sparse_categorical_accuracy: 0.909 - ETA: 0s - loss: 0.2919 - sparse_categorical_accuracy: 0.909 - ETA: 0s - loss: 0.2920 - sparse_categorical_accuracy: 0.909 - ETA: 0s - loss: 0.2919 - sparse_categorical_accuracy: 0.909 - ETA: 0s - loss: 0.2920 - 

 128/1325 [=>............................] - ETA: 3:06:01 - loss: 0.1940 - sparse_categorical_accuracy: 0.96 - ETA: 1:34:21 - loss: 0.1644 - sparse_categorical_accuracy: 0.95 - ETA: 1:03:31 - loss: 0.1874 - sparse_categorical_accuracy: 0.94 - ETA: 48:03 - loss: 0.1606 - sparse_categorical_accuracy: 0.9570 - ETA: 39:11 - loss: 0.1536 - sparse_categorical_accuracy: 0.95 - ETA: 32:59 - loss: 0.1593 - sparse_categorical_accuracy: 0.95 - ETA: 28:32 - loss: 0.1623 - sparse_categorical_accuracy: 0.95 - ETA: 25:12 - loss: 0.1664 - sparse_categorical_accuracy: 0.95 - ETA: 22:35 - loss: 0.1624 - sparse_categorical_accuracy: 0.95 - ETA: 20:30 - loss: 0.1742 - sparse_categorical_accuracy: 0.94 - ETA: 18:46 - loss: 0.1851 - sparse_categorical_accuracy: 0.94 - ETA: 17:18 - loss: 0.1834 - sparse_categorical_accuracy: 0.94 - ETA: 16:05 - loss: 0.1838 - sparse_categorical_accuracy: 0.94 - ETA: 15:03 - loss: 0.1852 - sparse_categorical_accuracy: 0.94 - ETA: 14:11 - loss: 0.1973 - sparse_categorical_accu

...

 128/1325 [=>............................] - ETA: 2:31:29 - loss: 0.0145 - sparse_categorical_accuracy: 1.00 - ETA: 1:16:41 - loss: 0.0354 - sparse_categorical_accuracy: 0.99 - ETA: 51:42 - loss: 0.0403 - sparse_categorical_accuracy: 0.9896 - ETA: 39:11 - loss: 0.0330 - sparse_categorical_accuracy: 0.99 - ETA: 31:39 - loss: 0.0374 - sparse_categorical_accuracy: 0.99 - ETA: 26:37 - loss: 0.0388 - sparse_categorical_accuracy: 0.98 - ETA: 23:02 - loss: 0.0397 - sparse_categorical_accuracy: 0.98 - ETA: 20:18 - loss: 0.0386 - sparse_categorical_accuracy: 0.99 - ETA: 18:13 - loss: 0.0419 - sparse_categorical_accuracy: 0.98 - ETA: 16:33 - loss: 0.0474 - sparse_categorical_accuracy: 0.98 - ETA: 15:12 - loss: 0.0439 - sparse_categorical_accuracy: 0.98 - ETA: 14:02 - loss: 0.0417 - sparse_categorical_accuracy: 0.98 - ETA: 13:05 - loss: 0.0439 - sparse_categorical_accuracy: 0.98 - ETA: 12:15 - loss: 0.0423 - sparse_categorical_accuracy: 0.99 - ETA: 11:32 - loss: 0.0404 - sparse_categorical_accura

1325/1325 [==============================] - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0345 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0346 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0347 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0347 - sparse_categorical_accuracy: 0.989 - ETA: 0s - loss: 0.0347 - 

 128/1325 [=>............................] - ETA: 2:36:09 - loss: 0.0048 - sparse_categorical_accuracy: 1.00 - ETA: 1:19:12 - loss: 0.0213 - sparse_categorical_accuracy: 0.99 - ETA: 53:33 - loss: 0.0466 - sparse_categorical_accuracy: 0.9896 - ETA: 40:35 - loss: 0.0600 - sparse_categorical_accuracy: 0.98 - ETA: 32:44 - loss: 0.0693 - sparse_categorical_accuracy: 0.98 - ETA: 27:32 - loss: 0.0669 - sparse_categorical_accuracy: 0.98 - ETA: 23:48 - loss: 0.0638 - sparse_categorical_accuracy: 0.98 - ETA: 21:01 - loss: 0.0568 - sparse_categorical_accuracy: 0.98 - ETA: 18:49 - loss: 0.0567 - sparse_categorical_accuracy: 0.98 - ETA: 17:05 - loss: 0.0517 - sparse_categorical_accuracy: 0.98 - ETA: 15:41 - loss: 0.0486 - sparse_categorical_accuracy: 0.98 - ETA: 14:29 - loss: 0.0456 - sparse_categorical_accuracy: 0.98 - ETA: 13:28 - loss: 0.0516 - sparse_categorical_accuracy: 0.98 - ETA: 12:36 - loss: 0.0482 - sparse_categorical_accuracy: 0.98 - ETA: 11:53 - loss: 0.0454 - sparse_categorical_accura

 128/1325 [=>............................] - ETA: 2:17:08 - loss: 0.0072 - sparse_categorical_accuracy: 1.00 - ETA: 1:09:30 - loss: 0.0115 - sparse_categorical_accuracy: 1.00 - ETA: 47:02 - loss: 0.0279 - sparse_categorical_accuracy: 0.9948 - ETA: 35:40 - loss: 0.0220 - sparse_categorical_accuracy: 0.99 - ETA: 28:52 - loss: 0.0179 - sparse_categorical_accuracy: 0.99 - ETA: 24:17 - loss: 0.0213 - sparse_categorical_accuracy: 0.99 - ETA: 21:06 - loss: 0.0188 - sparse_categorical_accuracy: 0.99 - ETA: 18:37 - loss: 0.0181 - sparse_categorical_accuracy: 0.99 - ETA: 16:45 - loss: 0.0165 - sparse_categorical_accuracy: 0.99 - ETA: 15:13 - loss: 0.0151 - sparse_categorical_accuracy: 0.99 - ETA: 14:01 - loss: 0.0153 - sparse_categorical_accuracy: 0.99 - ETA: 12:57 - loss: 0.0186 - sparse_categorical_accuracy: 0.99 - ETA: 12:05 - loss: 0.0189 - sparse_categorical_accuracy: 0.99 - ETA: 11:19 - loss: 0.0192 - sparse_categorical_accuracy: 0.99 - ETA: 10:39 - loss: 0.0181 - sparse_categorical_accura

1325/1325 [==============================] - ETA: 1s - loss: 0.0291 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0291 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0291 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0291 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0291 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 1s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0293 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0292 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0293 - sparse_categorical_accuracy: 0.991 - ETA: 0s - loss: 0.0293 - 

In [34]:
ans = model.predict_generator(valGen)

In [36]:
val_predict = np.argmax(ans, axis=1)
val_predict

array([ 0,  0, 14, ...,  2, 15,  0], dtype=int64)

In [37]:
type(valGen.labels)

dict

In [30]:
len(load_model.layers)

20

# Transfer Learning Test

In [12]:
# 加载含有自定义层的模型
load_model = keras.models.load_model('.\model-NewModelCpu_10.h5',
                                     custom_objects={'Melspectrogram': Melspectrogram,
                                                     'Normalization2D': Normalization2D})

In [32]:
# =================================================================
# 迁移学习测试
# =================================================================
x = load_model.layers[18].output
prediction = keras.layers.Dense(21, activation='softmax', name='new_dense')(x)
# 创建迁移学习模型
transfer_model = Model(input=load_model.input, output=prediction)
# 编译新模型
transfer_model.compile(optimizer='adam',
                       loss=['sparse_categorical_crossentropy'],
                       metrics=['sparse_categorical_accuracy'])


D:\SoftWare\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ne...)`
  after removing the cwd from sys.path.


In [43]:
# transfer_model.summary()

# ResNet+BiLSTM+Attention

In [6]:
# =================================================================
# 残差网络
# =================================================================
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    Returns:
    block -- string/character, used to name the layers, depending on their position in the network X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    # defining name basis
    conv_name_base = 'CNN-' + str(stage) + '-' + block
    bn_name_base = 'BN-' + str(stage) + '-' + block    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X
    with tf.name_scope('ResBlock_'+str(stage)):
        # First component of main path
        X = Conv2D(filters=F1, kernel_size=(3, 3), strides=(1, 1), padding='same', name=conv_name_base+'-a')(X)
        X = BatchNormalization(name=bn_name_base + '-a')(X)
        X = Activation('relu')(X)

        # Second component of main path
        X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '-b')(X)
        X = BatchNormalization(name=bn_name_base + '-b')(X)
        X = Activation('relu')(X)

        # Third component of main path
        # botteneck 
        X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='same', name=conv_name_base+'-c')(X)
        X = BatchNormalization(name=bn_name_base + '-c')(X)

        # 残差实现
        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X = Add()([X, X_shortcut])
        X = Activation('relu')(X)
    return X


In [7]:
# tf.reset_default_graph()
iLen = 16000
sr = 16000
inputs = Input((iLen,))
x = Reshape((1, -1))(inputs)
x = Melspectrogram(n_dft=1024, n_hop=128, input_shape=(1, iLen),
                   padding='same', sr=sr, n_mels=80,
                   fmin=40.0, fmax=sr/2,
# ``2.0`` to get power-spectrogram, ``1.0`` to get amplitude-spectrogram.                   
                   power_melgram=1.0,
                   return_decibel_melgram=True,
                   trainable_fb=False, trainable_kernel=False,
                   name='mel_stft')(x)
x = Normalization2D(int_axis=0, name='Normalization2D')(x)
x = Permute((2, 1, 3))(x)  # (125*80*1)

# DenseBlock 1
x = identity_block(x, 3, [32, 32, 1], 1, 'block')
x = identity_block(x, 3, [32, 32, 1], 2, 'block')
x = MaxPooling2D(pool_size=[2, 2], strides=[1, 1])(x)

# 在索引 axis 轴，移除 1 个尺寸的维度。
x = Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)
x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)   
x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)  

xFirst = Lambda(lambda q: q[:, 64])(x)
query = Dense(128)(xFirst)  # 128
# dot product attention 计算点积(注意力机制)
# ([128],[125,128]) = 125 相当于对每一个时刻进行打分
attScores = Dot(axes=[1, 2])([query, x])
# [b_s, seq_len]
attScores = Softmax(name='attSoftmax')(attScores)

# rescale sequence
# axes: 整数或者整数元组， 一个或者几个进行点积的轴。
# [b_s, vec_dim]    125, 125*128 = 128
attVector = Dot(axes=[1, 1])([attScores, x])
x = Dense(64, activation='relu')(attVector)
# # activation=None
# x = Dense(32)(x)
output = Dense(nCategs, activation='softmax', name='output')(x)
# x = Flatten()(x)
# output = Dense(21)(x)
model = Model(inputs=[inputs], outputs=[output])

# DenseNet

In [12]:

# # tf.reset_default_graph()
# iLen = 16000
# sr = 16000
# inputs = Input((iLen,))
# x = Reshape((1, -1))(inputs)
# x = Melspectrogram(n_dft=1024, n_hop=128, input_shape=(1, iLen),
#                    padding='same', sr=sr, n_mels=80,
#                    fmin=40.0, fmax=sr/2,
# # ``2.0`` to get power-spectrogram, ``1.0`` to get amplitude-spectrogram.                   
#                    power_melgram=1.0,
#                    return_decibel_melgram=True,
#                    trainable_fb=False, trainable_kernel=False,
#                    name='mel_stft')(x)
# x = Normalization2D(int_axis=0, name='Normalization2D')(x)
# x = Permute((2, 1, 3))(x)  # (125*80*1)

# # DenseBlock 1
# x = identity_block(x, 3, [32, 32, 1], 1, 'block')
# shortcut1 = x
# x = BatchNormalization()(x)

# # DenseBlock 2
# x = identity_block(x, 3, [32, 32, 1], 2, 'block')
# shortcut2 = x
# x = Add()([x, shortcut1])
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# # DenseBlock 3
# x = identity_block(x, 3, [32, 32, 1], 3, 'block')
# x = Add()([x, shortcut1])
# x = Add()([x, shortcut2])
# x = BatchNormalization()(x)
# x = Activation('relu')(x)


# # x = identity_block(x, 3, [32, 32, 1], 4, 'block')

# # x = identity_block(x, 3, [32, 32, 1], 5, 'block')
# # x = identity_block(x, 3, [32, 32, 1], 6, 'block')

# x = MaxPooling2D(pool_size=[2, 2], strides=[1, 1])(x)

# # 在索引 axis 轴，移除 1 个尺寸的维度。
# x = Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)
# x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)   
# x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)  

# xFirst = Lambda(lambda q: q[:, 64])(x)
# query = Dense(128)(xFirst)  # 128
# # dot product attention 计算点积(注意力机制)
# # ([128],[125,128]) = 125 相当于对每一个时刻进行打分
# attScores = Dot(axes=[1, 2])([query, x])
# # [b_s, seq_len]
# attScores = Softmax(name='attSoftmax')(attScores)

# # rescale sequence
# # axes: 整数或者整数元组， 一个或者几个进行点积的轴。
# # [b_s, vec_dim]    125, 125*128 = 128
# attVector = Dot(axes=[1, 1])([attScores, x])
# x = Dense(64, activation='relu')(attVector)
# # # activation=None
# # x = Dense(32)(x)
# output = Dense(nCategs, activation='softmax', name='output')(x)
# # x = Flatten()(x)
# # output = Dense(21)(x)
# model = Model(inputs=[inputs], outputs=[output])

In [10]:
model.compile(optimizer='adam',
              loss=['sparse_categorical_crossentropy'],
              metrics=['sparse_categorical_accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16000)        0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 16000)     0           input_1[0][0]                    
__________________________________________________________________________________________________
mel_stft (Melspectrogram)       (None, 80, 125, 1)   1091664     reshape_1[0][0]                  
__________________________________________________________________________________________________
Normalization2D (Normalization2 (None, 80, 125, 1)   0           mel_stft[0][0]                   
__________________________________________________________________________________________________
permute_1 

In [10]:
tensorboard = TensorBoard(log_dir='./boards/Dense-ResNet/v1_12cmd',
                          histogram_freq=0,
                          write_graph=True,  # 是否存储网络结构图
                          write_grads=True, # 是否可视化梯度直方图
                          write_images=True,# 是否可视化参数
                          embeddings_freq=0, 
                          embeddings_layer_names=None, 
                          embeddings_metadata=None)
earlystopper = EarlyStopping(monitor='val_sparse_categorical_accuracy',
                             patience=7, verbose=2)
checkpointer = ModelCheckpoint("./models/Dense-ResNet-AttLSTM-12cmd-v1-{epoch:02d}.h5",
                               monitor='val_sparse_categorical_accuracy',
                               verbose=1, save_best_only=True)

In [11]:
lrate = LearningRateScheduler(step_decay)
results = model.fit_generator(trainGen, validation_data=valGen,
                              epochs=40, use_multiprocessing=True,
                              workers=4,
                              callbacks=[earlystopper, checkpointer,
                                         lrate, tensorboard])

Epoch 1/40
Changing learning rate to 0.001


127/798 [===>..........................] - ETA: 2:55:45 - loss: 2.6531 - sparse_categorical_accuracy: 0.01 - ETA: 1:29:10 - loss: 2.5708 - sparse_categorical_accuracy: 0.01 - ETA: 1:00:18 - loss: 2.4893 - sparse_categorical_accuracy: 0.21 - ETA: 45:52 - loss: 2.3981 - sparse_categorical_accuracy: 0.3438 - ETA: 37:12 - loss: 2.3361 - sparse_categorical_accuracy: 0.39 - ETA: 31:25 - loss: 2.2891 - sparse_categorical_accuracy: 0.41 - ETA: 27:18 - loss: 2.1998 - sparse_categorical_accuracy: 0.45 - ETA: 24:12 - loss: 2.1095 - sparse_categorical_accuracy: 0.48 - ETA: 21:47 - loss: 2.0343 - sparse_categorical_accuracy: 0.50 - ETA: 19:52 - loss: 2.0138 - sparse_categorical_accuracy: 0.51 - ETA: 18:17 - loss: 1.9839 - sparse_categorical_accuracy: 0.51 - ETA: 16:59 - loss: 1.9406 - sparse_categorical_accuracy: 0.52 - ETA: 15:52 - loss: 1.8809 - sparse_categorical_accuracy: 0.54 - ETA: 14:54 - loss: 1.8709 - sparse_categorical_accuracy: 0.54 - ETA: 14:05 - loss: 1.8555 - sparse_categorical_accura

383/798 [=============>................] - ETA: 2:54 - loss: 1.2889 - sparse_categorical_accuracy: 0.648 - ETA: 2:54 - loss: 1.2876 - sparse_categorical_accuracy: 0.648 - ETA: 2:53 - loss: 1.2857 - sparse_categorical_accuracy: 0.648 - ETA: 2:53 - loss: 1.2838 - sparse_categorical_accuracy: 0.649 - ETA: 2:52 - loss: 1.2817 - sparse_categorical_accuracy: 0.649 - ETA: 2:52 - loss: 1.2804 - sparse_categorical_accuracy: 0.649 - ETA: 2:51 - loss: 1.2791 - sparse_categorical_accuracy: 0.649 - ETA: 2:51 - loss: 1.2766 - sparse_categorical_accuracy: 0.650 - ETA: 2:50 - loss: 1.2753 - sparse_categorical_accuracy: 0.650 - ETA: 2:49 - loss: 1.2735 - sparse_categorical_accuracy: 0.650 - ETA: 2:49 - loss: 1.2722 - sparse_categorical_accuracy: 0.651 - ETA: 2:48 - loss: 1.2711 - sparse_categorical_accuracy: 0.651 - ETA: 2:48 - loss: 1.2701 - sparse_categorical_accuracy: 0.651 - ETA: 2:47 - loss: 1.2681 - sparse_categorical_accuracy: 0.651 - ETA: 2:47 - loss: 1.2658 - sparse_categorical_accuracy: 0.652

511/798 [==================>...........] - ETA: 2:17 - loss: 1.1284 - sparse_categorical_accuracy: 0.678 - ETA: 2:17 - loss: 1.1270 - sparse_categorical_accuracy: 0.678 - ETA: 2:16 - loss: 1.1259 - sparse_categorical_accuracy: 0.679 - ETA: 2:16 - loss: 1.1244 - sparse_categorical_accuracy: 0.679 - ETA: 2:16 - loss: 1.1229 - sparse_categorical_accuracy: 0.679 - ETA: 2:15 - loss: 1.1221 - sparse_categorical_accuracy: 0.679 - ETA: 2:15 - loss: 1.1212 - sparse_categorical_accuracy: 0.680 - ETA: 2:14 - loss: 1.1202 - sparse_categorical_accuracy: 0.680 - ETA: 2:14 - loss: 1.1190 - sparse_categorical_accuracy: 0.680 - ETA: 2:13 - loss: 1.1176 - sparse_categorical_accuracy: 0.680 - ETA: 2:13 - loss: 1.1160 - sparse_categorical_accuracy: 0.681 - ETA: 2:12 - loss: 1.1146 - sparse_categorical_accuracy: 0.681 - ETA: 2:12 - loss: 1.1137 - sparse_categorical_accuracy: 0.681 - ETA: 2:11 - loss: 1.1129 - sparse_categorical_accuracy: 0.682 - ETA: 2:11 - loss: 1.1116 - sparse_categorical_accuracy: 0.682

640/798 [=======================>......] - ETA: 1:27 - loss: 1.0038 - sparse_categorical_accuracy: 0.707 - ETA: 1:26 - loss: 1.0029 - sparse_categorical_accuracy: 0.708 - ETA: 1:26 - loss: 1.0023 - sparse_categorical_accuracy: 0.708 - ETA: 1:26 - loss: 1.0011 - sparse_categorical_accuracy: 0.708 - ETA: 1:25 - loss: 1.0001 - sparse_categorical_accuracy: 0.708 - ETA: 1:25 - loss: 0.9990 - sparse_categorical_accuracy: 0.709 - ETA: 1:25 - loss: 0.9980 - sparse_categorical_accuracy: 0.709 - ETA: 1:24 - loss: 0.9972 - sparse_categorical_accuracy: 0.709 - ETA: 1:24 - loss: 0.9963 - sparse_categorical_accuracy: 0.709 - ETA: 1:24 - loss: 0.9950 - sparse_categorical_accuracy: 0.710 - ETA: 1:23 - loss: 0.9943 - sparse_categorical_accuracy: 0.710 - ETA: 1:23 - loss: 0.9937 - sparse_categorical_accuracy: 0.710 - ETA: 1:22 - loss: 0.9926 - sparse_categorical_accuracy: 0.710 - ETA: 1:22 - loss: 0.9917 - sparse_categorical_accuracy: 0.711 - ETA: 1:22 - loss: 0.9908 - sparse_categorical_accuracy: 0.711

...

383/798 [=============>................] - ETA: 2:10 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:09 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:09 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:09 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:08 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:08 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:08 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:08 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:07 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:07 - loss: 0.0042 - sparse_categorical_accuracy: 0.999 - ETA: 2:07 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:06 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:06 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:06 - loss: 0.0043 - sparse_categorical_accuracy: 0.999 - ETA: 2:06 - loss: 0.0043 - sparse_categorical_accuracy: 0.999

511/798 [==================>...........] - ETA: 1:36 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:36 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:36 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:35 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:35 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:35 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:35 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:34 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:34 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:34 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:34 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:33 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:33 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:33 - loss: 0.0041 - sparse_categorical_accuracy: 0.999 - ETA: 1:33 - loss: 0.0041 - sparse_categorical_accuracy: 0.999

642/798 [=======================>......] - ETA: 1:05 - loss: 0.0040 - sparse_categorical_accuracy: 0.999 - ETA: 1:05 - loss: 0.0040 - sparse_categorical_accuracy: 0.999 - ETA: 1:05 - loss: 0.0040 - sparse_categorical_accuracy: 0.999 - ETA: 1:05 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:04 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:04 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:04 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:04 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:03 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:03 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:03 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:03 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:02 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:02 - loss: 0.0039 - sparse_categorical_accuracy: 0.999 - ETA: 1:02 - loss: 0.0039 - sparse_categorical_accuracy: 0.999

774/798 [============================>.] - ETA: 35s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 34s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 34s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 34s - loss: 0.0039 - sparse_categorical_accuracy: 0.99 - ETA: 34s - loss: 0.0039 - sparse_categorical_accuracy: 0.99 - ETA: 34s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 33s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 33s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 33s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 33s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 32s - loss: 0.0040 - sparse_categorical_accuracy: 0.99 - ETA: 32s - loss: 0.0039 - sparse_categorical_accuracy: 0.99 - ETA: 32s - loss: 0.0039 - sparse_categorical_accuracy: 0.99 - ETA: 32s - loss: 0.0039 - sparse_categorical_accuracy: 0.99 - ETA: 32s - loss: 0.0039 - sparse_categorical_accuracy: 0.99 - ETA: 31s - loss: 0.0039 - s

798/798 [==============================] - ETA: 5s - loss: 0.0037 - sparse_categorical_accuracy: 0.999 - ETA: 4s - loss: 0.0037 - sparse_categorical_accuracy: 0.999 - ETA: 4s - loss: 0.0037 - sparse_categorical_accuracy: 0.999 - ETA: 4s - loss: 0.0037 - sparse_categorical_accuracy: 0.999 - ETA: 4s - loss: 0.0037 - sparse_categorical_accuracy: 0.999 - ETA: 4s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 3s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 3s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 3s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 3s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 2s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 2s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 2s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 2s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 2s - loss: 0.0036 - sparse_categorical_accuracy: 0.999 - ETA: 1s - loss: 0.0036 - sp

In [12]:
loss, acc = model.evaluate_generator(valGen)
print('loss:{}'.format(loss), 
      'accuracy:{}'.format(acc))

loss:0.1270898375055265 accuracy:0.9721554487179487


In [12]:
predictions = model.predict_generator(valGen)

In [15]:
predictions.shape

(9920,)

In [14]:
predictions = np.argmax(predictions, axis=1)

In [16]:
labels = []
for v in valGen.labels.values():
    labels.append(v)
print(len(labels))
print(len(predictions))

9981
9920


In [7]:
# 加载含有自定义层的模型
load_model = keras.models.load_model('.\model-ResNet-12cmd-test1.h5',
                                     custom_objects={'Melspectrogram': Melspectrogram,
                                                     'Normalization2D': Normalization2D})

In [9]:
predictions = np.argmax(load_model.predict_generator(valGen), axis=1)

In [16]:
loss, acc = load_model.evaluate_generator(valGen)
print('loss:{}'.format(loss), 
      'accuracy:{}'.format(acc))

loss:0.09772179100792318 accuracy:0.97265625


In [16]:
cmd_12 = [
    'unknown', 
#     'silence',
    'yes', 'no', 'up', 'down', 'left',
    'right', 'on', 'off', 'stop', 'go'
]

In [17]:
cmd_21 = ['unknown', 'nine', 'yes', 'no', 'up', 'down', 'left',
          'right', 'on', 'off', 'stop', 'go', 'zero', 'one', 'two',
          'three', 'four', 'five', 'six', 'seven', 'eight']

In [17]:
word_35 = [
#     'unknown',
    'nine', 'yes', 'no', 'up', 'down', 'left',
    'right', 'on', 'off', 'stop', 'go', 'zero', 'one', 'two',
    'three', 'four', 'five', 'six', 'seven', 'eight',
    'backward',
    'bed', 'bird', 'cat', 'dog',
    'follow', 'forward', 'happy',
    'house',
    'learn',
    'marvin', 'sheila', 'tree', 'visual', 'wow'
]

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_true=labels[:9920],
                            y_pred=predictions,
                            target_names=word_35
                           ))

              precision    recall  f1-score   support

        nine       0.96      0.97      0.96       356
         yes       0.96      0.97      0.97       397
          no       0.97      0.93      0.95       406
          up       0.92      0.93      0.92       350
        down       0.96      0.93      0.95       377
        left       0.96      0.95      0.95       352
       right       0.98      0.97      0.98       363
          on       0.96      0.94      0.95       363
         off       0.95      0.93      0.94       373
        stop       0.97      0.98      0.98       350
          go       0.93      0.97      0.94       372
        zero       0.96      0.97      0.97       384
         one       0.97      0.94      0.96       351
         two       0.96      0.97      0.96       345
       three       0.93      0.94      0.94       356
        four       0.87      0.99      0.92       312
        five       0.96      0.96      0.96       367
         six       0.96    

In [1]:
# [k for k, v in valGen.labels.items() if v==0]